In [1]:
import numpy as np
import pandas as pd

In [3]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
 
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
 
# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
 
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
 

# install findspark using pip
!pip install -q findspark

In [4]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 52kB/s 
     |████████████████████████████████| 204kB 41.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=19e2083990ff65604d928cdb5f35ae8105bc7dcb2d28104f87514555aa3e466e
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [5]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.ml.linalg import Vectors
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
binance = np.load('/content/binance_order_array.npy', allow_pickle=True)
binance

array([{'lastUpdateId': 2041640431, 'bids': [['0.02703200', '3.37300000'], ['0.02702800', '4.00000000'], ['0.02702500', '0.61800000'], ['0.02702400', '5.83200000'], ['0.02702300', '3.11700000'], ['0.02702200', '0.28600000'], ['0.02702100', '3.07500000'], ['0.02701600', '9.71200000'], ['0.02701500', '25.76300000'], ['0.02701400', '3.06800000'], ['0.02700700', '6.50000000'], ['0.02700600', '6.19900000'], ['0.02700500', '3.04000000'], ['0.02700300', '4.40500000'], ['0.02700000', '7.58700000'], ['0.02699800', '0.03800000'], ['0.02699700', '5.41800000'], ['0.02699600', '5.57400000'], ['0.02699200', '9.58000000'], ['0.02698900', '0.01000000'], ['0.02698700', '3.05400000'], ['0.02698600', '2.50000000'], ['0.02698500', '0.22300000'], ['0.02698000', '0.10200000'], ['0.02697900', '7.38700000'], ['0.02697800', '7.38000000'], ['0.02697700', '6.16800000'], ['0.02697600', '2.50000000'], ['0.02697400', '5.95000000'], ['0.02697300', '0.05000000'], ['0.02696800', '27.80800000'], ['0.02696600', '5.55900

In [9]:
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)
rdd = ssc.sparkContext.parallelize(binance)

In [10]:
rdd = rdd.map(lambda x: (x['time'], x['bids'], x['asks']))

In [11]:
rdd.take(2)

[(1609195948717,
  [['0.02703200', '3.37300000'],
   ['0.02702800', '4.00000000'],
   ['0.02702500', '0.61800000'],
   ['0.02702400', '5.83200000'],
   ['0.02702300', '3.11700000'],
   ['0.02702200', '0.28600000'],
   ['0.02702100', '3.07500000'],
   ['0.02701600', '9.71200000'],
   ['0.02701500', '25.76300000'],
   ['0.02701400', '3.06800000'],
   ['0.02700700', '6.50000000'],
   ['0.02700600', '6.19900000'],
   ['0.02700500', '3.04000000'],
   ['0.02700300', '4.40500000'],
   ['0.02700000', '7.58700000'],
   ['0.02699800', '0.03800000'],
   ['0.02699700', '5.41800000'],
   ['0.02699600', '5.57400000'],
   ['0.02699200', '9.58000000'],
   ['0.02698900', '0.01000000'],
   ['0.02698700', '3.05400000'],
   ['0.02698600', '2.50000000'],
   ['0.02698500', '0.22300000'],
   ['0.02698000', '0.10200000'],
   ['0.02697900', '7.38700000'],
   ['0.02697800', '7.38000000'],
   ['0.02697700', '6.16800000'],
   ['0.02697600', '2.50000000'],
   ['0.02697400', '5.95000000'],
   ['0.02697300', '0.0500

In [12]:
def min_max(iterator):

    max_b = []
    min_a = []

    for x in iterator:

        bids = []
        asks = []


        for i in range(100):

            bids.append(x[1][i][0])
            asks.append(x[2][i][0])
          
        max_b.append([x[0],max(bids),min(asks)] )

    return max_b

In [13]:
final = rdd.mapPartitions(min_max)

In [14]:
final.take(5)

[[1609195948717, '0.02703200', '0.02703300'],
 [1609195950271, '0.02703200', '0.02703300'],
 [1609195951836, '0.02703500', '0.02704600'],
 [1609195953400, '0.02704500', '0.02704600'],
 [1609195954952, '0.02704500', '0.02704600']]

In [15]:
!pip install influxdb

     |████████████████████████████████| 81kB 3.7MB/s 


In [16]:
!sudo apt-get update && sudo apt-get install influxdb

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [18]:
!sudo service influxdb start

 * Starting database influxd
   ...done.


In [19]:
from influxdb import InfluxDBClient
client = InfluxDBClient(host='localhost', port=8086)

In [20]:
client.create_database('binance_order')

In [21]:
client.get_list_database()

[{'name': '_internal'}, {'name': 'binance_order'}]

In [27]:
def influxCon(partitions):
    result = []
    for i in range(np.array(partitions).shape[0] - 1):
        result.append('binance_order date='+str(partitions[i][0])+','+'bids='+ partitions[i][1]+','+'asks='+ partitions[i][1])
    return result

In [28]:
inf_bin=final.mapPartitions(influxCon)  

In [29]:
inf_bin.take(2)

['binance_order date=1609195948717,bids=0.02703200,asks=0.02703200',
 'binance_order date=1609195950271,bids=0.02703200,asks=0.02703200']

In [30]:
client.write_points(inf_bin.collect(), database='binance_order', time_precision='ms', batch_size=1, protocol='line')

True

In [31]:
client.switch_database('binance_order')

In [32]:
q='select * from binance_order.autogen.binance_order'
bin_order = pd.DataFrame(client.query(q).get_points())

In [33]:
bin_order

,time,asks,bids,date
0,2021-01-03T16:31:52.553Z,0.027032,0.027032,1.609196e+12
1,2021-01-03T16:31:52.569Z,0.027032,0.027032,1.609196e+12
2,2021-01-03T16:31:52.576Z,0.027035,0.027035,1.609196e+12
3,2021-01-03T16:31:52.583Z,0.027045,0.027045,1.609196e+12
4,2021-01-03T16:31:52.591Z,0.027045,0.027045,1.609196e+12
...,...,...,...,...
10045,2021-01-03T16:33:07.329Z,0.027305,0.027305,1.609239e+12
10046,2021-01-03T16:33:07.336Z,0.027310,0.027310,1.609239e+12
10047,2021-01-03T16:33:07.344Z,0.027320,0.027320,1.609239e+12
10048,2021-01-03T16:33:07.351Z,0.027317,0.027317,1.609239e+12
